# Define parameters

In [1]:
%matplotlib inline

import numpy as np
from essentials.physical_models import VdP
from essentials.plotResults import plot_timeseries, plot_parameters
from essentials.create import create_truth, create_ensemble
from essentials.DA import dataAssimilation



rng = np.random.default_rng(0)

dt_t = 2e-4

true_params = dict(model=VdP,
                   t_start=0.5,
                   t_stop=1.,
                   t_max=1.5,
                   Nt_obs=30,
                   dt=dt_t,
                   psi0=rng.random(2)+5,
                   std_obs=0.2,
                   noise_type='gauss,additive'
                   )

forecast_params = dict(filter='EnKF',
                       m=10,
                       dt=dt_t,
                       model=VdP,
                       est_a=dict(zeta=(40, 80.)
                                 ),
                       std_psi=0.3,
                       alpha_distr='uniform',
                       inflation=1.01
                       )




# Run simulation

In [2]:



truth = create_truth(**true_params)
y_obs, t_obs = [truth[key].copy() for key in ['y_obs', 't_obs']]

ensemble = create_ensemble(**forecast_params)
filter_ens = dataAssimilation(ensemble.copy(), y_obs=y_obs, t_obs=t_obs, std_obs=0.01)

# Plot results

In [3]:

# Forecast the ensemble further without assimilation
ens = filter_ens.copy()
psi, t = ens.time_integrate(int(4 * true_params['Nt_obs']), averaged=False)
ens.update_history(psi, t)

plot_timeseries(ens, truth, plot_ensemble_members=False)
    
plot_parameters(ens, truth, reference_p=truth['case'].alpha0)
